In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import spacy

from pdfminer.high_level import extract_text
# import pdfplumber
import re
import pandas as pd
import itertools
from sentence_transformers import SentenceTransformer, util

In [2]:
# For BERT based NER
# tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
# model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

In [3]:
# For spaCy based NER

# !python -m spacy download en_core_web_trf
import spacy_transformers
nlp_spacy_trf = spacy.load("en_core_web_trf")
nlp_spacy_trf.add_pipe("merge_noun_chunks")
# model = SentenceTransformer('distilbert-base-nli-mean-tokens')

<function spacy.pipeline.functions.merge_noun_chunks(doc: spacy.tokens.doc.Doc) -> spacy.tokens.doc.Doc>

In [3]:
import en_core_web_trf
nlp_spacy_trf = en_core_web_trf.load()
nlp_spacy_trf.add_pipe("merge_noun_chunks")

<function spacy.pipeline.functions.merge_noun_chunks(doc: spacy.tokens.doc.Doc) -> spacy.tokens.doc.Doc>

In [4]:
# sample text 
text = "If you could talk us through, how is the truck profitability looking like, purely from a gross margin perspective, I understand there will be operating effects there." 

# create a spaCy object 
doc = nlp_spacy_trf(text)

for phrase in doc:
    if phrase.pos_ == "NOUN":
        print("------------")
        print(phrase.text)
        print(phrase.tag_)
        print(phrase.pos_)
        print("------------")


------------
the truck profitability
NN
NOUN
------------
------------
a gross margin perspective
NN
NOUN
------------
------------
operating effects
NNS
NOUN
------------


/home/aum/Desktop/projects/con_trans_summary/con_summary_env/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


#### pdfminer.six

In [4]:
a = nlp_spacy_trf('Hi my name is XYZ')
for _ in a.sents:
    print(_)

Hi my name is XYZ


/home/aum/Desktop/projects/con_trans_summary/con_summary_env/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [17]:
# text = extract_text('../data/ashok_leyland_concall_transcript.pdf')
text = extract_text('../data/AGM_Transcript_Revised.pdf')

#### pdfplumber: Better results in pdf extraction -> Check later

with pdfplumber.open("../data/AGM_Transcript_Revised.pdf") as file:
    text = file.pages[10].extract_text()

In [ ]:
# Idea -> Creating tokens as Points of Interest (POI) in the text extracted from the PDF
# and using these tokens to extract Entities of Interest (EOI)
string = 'I now invite Mr. A.V. Mani Sundaram, CLID IN30163741521740.<mask>A.V. Mani Sundaram:'
rege = r'(<mask>(.*)<mask>(.*?):)|(<mask>(.*?):)'

re.findall(rege, string)[0][-1]

In [9]:
sentences = [
    " Thank you very much. We will now begin the question and answer session. We will wait for a moment while the question can be assembled. ",
    " Dear Shareholders, good morning and a very warm welcome to the Annual General Meeting of City   Union   Bank   Limited   for   FY'2021   through   video   conferencing   or   other   audio-visual facility. As a reminder, for the smooth conduct of the meeting, the members will be in the mute mode and audio and video will be opened when they will speak at the AGM as per the pre-registration. Please note that as per the requirements, the proceedings of the annual general meeting will be recorded  and  available on the website  of the Bank.  I now hand over the proceedings to Shri R. Mohan, the Chairman of City Union Bank Limited. Thank you and over ",
    "ask, question and answer",
    ]
sentence_embeddings = model.encode(sentences)

for sentence, embedding in zip(sentences, sentence_embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

# doc1 = nlp_spacy_trf(u)
# doc2 = nlp_spacy_trf(u'ask, question and answer')
print(util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[1]))
print(util.pytorch_cos_sim(sentence_embeddings[1], sentence_embeddings[2]))
print(util.pytorch_cos_sim(sentence_embeddings[0], sentence_embeddings[2]))

Sentence:  Thank you very much. We will now begin the question and answer session. We will wait for a moment while the question can be assembled. 
Embedding: [ 0.06344783  0.11630968  0.576895   -0.05625724 -0.38268468 -0.48887476
  0.01866261  0.22074395  1.7286305  -1.1689817   0.23807026  0.46779093
  0.6454454   0.04822326  0.6396784  -0.4240498  -0.37041032  0.05249115
 -0.37773827 -0.68010116  0.27323407 -0.3261881  -0.31360036  1.0595748
  0.5199989  -0.02574236  0.06939525  0.38551262 -0.05191303  0.8558748
  0.02803756 -0.2245875  -0.35413888  0.756421    0.1604854   0.04266265
  0.7750803  -0.16950706 -0.53763425 -0.20274737 -0.11885174 -0.61751074
  0.58929455 -0.3541158  -0.34435958 -0.22837614 -0.14803869 -0.04544735
 -1.3836113  -0.3095633  -0.8094896   0.27334154  0.16742463 -0.2938665
 -0.53182346 -0.38980478  0.33786342  0.7349423   0.71322507 -0.11272969
 -0.16684145  0.25240716 -0.1544134  -0.90846854  1.1012332  -0.2501768
  0.27942017 -0.02505153 -0.05847352 -0.914

#### NOTE: The processed data's accuracy will depend on the quality of pdf extraction 

In [18]:
# Step 1: Creating POI
text = text.replace('\n\n', '<mask>').replace('\n', '<m>').replace('\x0c', '')

In [38]:
# Step 2: Extracting EOI

all_eoi = re.findall(r'<mask>(.*?):', text)
all_eoi = [str(_.strip()) + ':' for _ in all_eoi]

list_of_entities = []
for entity in all_eoi:
    match_group = re.findall(r'(<mask>(.*)<mask>(.*?):)|(<mask>(.*?):)', entity)
    non_empty_group = [ele for inner_group in match_group for ele in inner_group if ele]
    # print(non_empty_group)
    if match_group:
        if len(non_empty_group[0]) < 2:
            pass
        else:
#             print(match_group[0][1])
            list_of_entities.append(non_empty_group[-1])
#     list_of_entities.extend(re.findall(r'<mask><mask>(.*)<mask><mask>(.*?):', entity)[-1])

# Approach: Preliminary weeding out entities based on length of the entity
# Assumption: A typical name would not exceed 40 characters
filtered_entity_list = [filtered_element for filtered_element in list_of_entities if len(filtered_element) <= 40]
print(filtered_entity_list)


['MANAGEMENT', 'BOARD OF DIRECTORS PRESENT', 'SPECIAL INVITES', 'Moderator', 'Shri R. Mohan', 'V. Ramesh', 'Shri R. Mohan', 'Moderator', 'Moderator', 'Aspi Bhesania', 'Moderator', 'A.V. Mani Sundaram', 'Moderator', 'J Abhishek', 'Moderator', 'G. Sankaran', 'Moderator', 'Santosh Kumar Saraf', 'Moderator', 'K.S. Balasubramanian', 'Moderator', 'Dr. N. Kamakodi', 'Shri R. Mohan', 'K. Vaidyanathan']


In [20]:
# Tentative Approach: Trying to filter out PER entities  (discarded)

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

for each_element in filtered_entity_list:
    ner = nlp(each_element.replace('.', ''))
#     print(ner)
# Cons: Does not work as expected; Moderator, Management does not fit under any entity; breaks down single words
# and provides entities (is this expected?) -> It is expected

NameError: name 'pipeline' is not defined

In [42]:
nlp_spacy_trf(" | ".join(filtered_entity_list)).ents

/home/aum/Desktop/projects/con_trans_summary/con_summary_env/lib/python3.9/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


(Shri R. Mohan,
 V. Ramesh,
 Shri R. Mohan,
 Aspi Bhesania,
 A.V. Mani Sundaram,
 J Abhishek,
 G. Sankaran,
 Santosh Kumar Saraf,
 K.S. Balasubramanian,
 N. Kamakodi,
 Shri R. Mohan,
 K. Vaidyanathan)

In [44]:
# Tentative Approach: Trying to filter out PER entities  (using spaCy based NEW approach)
# Selected approach

ner_list = []
for word_entity in nlp_spacy_trf(" | ".join(filtered_entity_list)).ents:
    print(word_entity.text)
    name_property = word_entity.text, word_entity.label_
    ner_list.append(name_property)
print(ner_list)

Shri R. Mohan
V. Ramesh
Shri R. Mohan
Aspi Bhesania
A.V. Mani Sundaram
J Abhishek
G. Sankaran
Santosh Kumar Saraf
K.S. Balasubramanian
N. Kamakodi
Shri R. Mohan
K. Vaidyanathan
[('Shri R. Mohan', 'PERSON'), ('V. Ramesh', 'PERSON'), ('Shri R. Mohan', 'PERSON'), ('Aspi Bhesania', 'PERSON'), ('A.V. Mani Sundaram', 'PERSON'), ('J Abhishek', 'PERSON'), ('G. Sankaran', 'PERSON'), ('Santosh Kumar Saraf', 'PERSON'), ('K.S. Balasubramanian', 'PERSON'), ('N. Kamakodi', 'PERSON'), ('Shri R. Mohan', 'PERSON'), ('K. Vaidyanathan', 'PERSON')]


In [35]:
# Step 3: Extracting location of all EOI

entity_span = []
for element in filtered_entity_list:
    for find_result in re.finditer(str(element) + ':', text):
        span = (element, int(find_result.start()), int(find_result.end()))
        entity_span.append(span)
entity_span = list(set(entity_span))
entity_span = sorted(entity_span, key=lambda span_list: span_list[1])
print(entity_span)

[('MANAGEMENT', 77, 88), ('BOARD OF DIRECTORS PRESENT', 222, 249), ('SPECIAL INVITES', 588, 604), ('Moderator', 799, 809), ('Shri R. Mohan', 1504, 1518), ('V. Ramesh', 3540, 3550), ('Shri R. Mohan', 4947, 4961), ('Moderator', 27547, 27557), ('Moderator', 27708, 27718), ('Aspi Bhesania', 27783, 27797), ('Moderator', 30098, 30108), ('A.V. Mani Sundaram', 30179, 30198), ('Moderator', 30729, 30739), ('J Abhishek', 30803, 30814), ('Moderator', 32751, 32761), ('G. Sankaran', 32948, 32960), ('Moderator', 37088, 37098), ('Santosh Kumar Saraf', 37170, 37190), ('Moderator', 37290, 37300), ('K.S. Balasubramanian', 37373, 37394), ('Moderator', 43089, 43099), ('Dr. N. Kamakodi', 43195, 43211), ('Shri R. Mohan', 70303, 70317), ('K. Vaidyanathan', 70483, 70499)]


In [45]:
# Step 4: Filtering out irrelevant EOI

filtered_entity_span = []
for entity_of_interest in entity_span:
    name = entity_of_interest[0].lower()
    for ner_result in ner_list:
        if ("management" in name) or (
            "moderator" in name) or (
            "invites" in name) or (
            "special" in name) or (
            "board" in name) or (
            "director" in name):
            filtered_entity_span.append(entity_of_interest)
            break
        elif (name in ner_result[0].lower()) or (ner_result[0].lower() in name):
            filtered_entity_span.append(entity_of_interest)
            break
        else:
            pass
print(filtered_entity_span)

[('MANAGEMENT', 77, 88), ('BOARD OF DIRECTORS PRESENT', 222, 249), ('SPECIAL INVITES', 588, 604), ('Moderator', 799, 809), ('Shri R. Mohan', 1504, 1518), ('V. Ramesh', 3540, 3550), ('Shri R. Mohan', 4947, 4961), ('Moderator', 27547, 27557), ('Moderator', 27708, 27718), ('Aspi Bhesania', 27783, 27797), ('Moderator', 30098, 30108), ('A.V. Mani Sundaram', 30179, 30198), ('Moderator', 30729, 30739), ('J Abhishek', 30803, 30814), ('Moderator', 32751, 32761), ('G. Sankaran', 32948, 32960), ('Moderator', 37088, 37098), ('Santosh Kumar Saraf', 37170, 37190), ('Moderator', 37290, 37300), ('K.S. Balasubramanian', 37373, 37394), ('Moderator', 43089, 43099), ('Dr. N. Kamakodi', 43195, 43211), ('Shri R. Mohan', 70303, 70317), ('K. Vaidyanathan', 70483, 70499)]


In [46]:
# Step 5: Extracting relevant information based on the EOI

desired_columns = ["Sr.No.", "Name", "GroupOfSentences"]
information_frame = pd.DataFrame(None, columns=desired_columns)
for sequence, entity_information in enumerate(filtered_entity_span):
    start_index = entity_information[1]
    end_index = entity_information[2]
    if entity_information[0].lower() == "management":
        management_string = text[
            end_index: filtered_entity_span[sequence+1][1]
        ].replace("<mask>", "|").replace("<m>", "|")
        management_list = management_string.split("|")
        
        frame = pd.DataFrame([[sequence, entity_information[0], management_string]], columns=desired_columns)

    elif ("directors" in entity_information[0].lower()) and ("board" in entity_information[0].lower()):
        director_string = text[
            end_index: filtered_entity_span[sequence+1][1]
        ].replace("<mask>", "|").replace("<m>", "|")
        director_list = director_string.split("|")

        frame = pd.DataFrame([[sequence, entity_information[0], director_string]], columns=desired_columns)

    else:
        try:
            relevant_string = text[
                end_index: filtered_entity_span[sequence+1][1]
            ].replace("<mask>", " ").replace("<m>", " ")
        except IndexError:
            relevant_string = text[
                end_index:
            ].replace("<mask>", " ").replace("<m>", " ")
    
        frame = pd.DataFrame([[sequence, entity_information[0], relevant_string]], columns=desired_columns)
    
    information_frame = pd.concat([information_frame, frame], axis=0)
    information_frame.reset_index(drop=True, inplace=True)

In [25]:
# information_frame.to_excel("../data/ashok_leyland_concall_transcript.xlsx", index=False)

In [47]:
information_frame

,Sr.No.,Name,GroupOfSentences
0,0,MANAGEMENT,SHRI R. MOHAN – NON-EXECUTIVE CHAIRMAN|DR. N....
1,1,BOARD OF DIRECTORS PRESENT,|SMT. ABARNA BHASKAR – INDEPENDENT DIRECTOR MR...
2,2,SPECIAL INVITES,"M/S. SUNDARAM & SRINIVASAN, STATUTORY CENTRA..."
3,3,Moderator,"Dear Shareholders, good morning and a very wa..."
4,4,Shri R. Mohan,Thank you! Let us commence this AGM with Inau...
5,5,V. Ramesh,"Hello. Good Morning. Dear members, you are re..."
6,6,Shri R. Mohan,"Thank you, Ramesh. CHAIRMAN SPEECH Esteemed ..."
7,7,Moderator,Thank you very much. We will now begin the qu...
8,8,Moderator,I now invite Mr. Aspi Bhesania CLID 120125000...
9,9,Aspi Bhesania,"Chairman Sir, I'm from Bombay. Sir Resolution..."


In [48]:
management_string

' SHRI R. MOHAN – NON-EXECUTIVE CHAIRMAN|DR. N. KAMAKODI – MANAGING DIRECTOR & CEO|MR. V. RAMESH – CFO & COMPANY SECRETARY|'

In [49]:
# Step 6: Identify conversations initiated by the Management

management_corpus = management_string.lower()

information_frame["type_of_conversation"] = None

question_count = 0
outlook_count = 0
for i, conversation_initiator in enumerate(information_frame["Name"].values):
    if conversation_initiator.strip().lower() in management_corpus:
        if question_count != 0:
            if outlook_count == 0:
                information_frame.loc[i, "type_of_conversation"] = "Outlook"
                outlook_count += 1
            else:
                information_frame.loc[i, "type_of_conversation"] = "Answer"
        else:
            information_frame.loc[i, "type_of_conversation"] = "Outlook"
            outlook_count += 1
            
    elif (conversation_initiator.strip().lower() != "management") and (
          conversation_initiator.strip().lower() != "moderator") and (
          "special invites" not in conversation_initiator.strip().lower()) and (
          "board" not in conversation_initiator.strip().lower()) and (
          "directors" not in conversation_initiator.strip().lower()):
        information_frame.loc[i, "type_of_conversation"] = "Question"
        question_count += 1

information_frame.loc[:information_frame[information_frame['type_of_conversation'] == "Outlook"].index.values.tolist()[0]-1, "type_of_conversation"] = None        

information_frame

,Sr.No.,Name,GroupOfSentences,type_of_conversation
0,0,MANAGEMENT,SHRI R. MOHAN – NON-EXECUTIVE CHAIRMAN|DR. N....,None
1,1,BOARD OF DIRECTORS PRESENT,|SMT. ABARNA BHASKAR – INDEPENDENT DIRECTOR MR...,None
2,2,SPECIAL INVITES,"M/S. SUNDARAM & SRINIVASAN, STATUTORY CENTRA...",None
3,3,Moderator,"Dear Shareholders, good morning and a very wa...",None
4,4,Shri R. Mohan,Thank you! Let us commence this AGM with Inau...,Outlook
5,5,V. Ramesh,"Hello. Good Morning. Dear members, you are re...",Outlook
6,6,Shri R. Mohan,"Thank you, Ramesh. CHAIRMAN SPEECH Esteemed ...",Outlook
7,7,Moderator,Thank you very much. We will now begin the qu...,None
8,8,Moderator,I now invite Mr. Aspi Bhesania CLID 120125000...,None
9,9,Aspi Bhesania,"Chairman Sir, I'm from Bombay. Sir Resolution...",Question


In [18]:
# Step 7: Filtering Answers and constructing a corpus

answer_corpus = [''.join(
    information_frame[
        information_frame["type_of_conversation"] == "Answer"
    ]["GroupOfSentences"].values.tolist()
)]

# Splitting the statements on a full stop (.)
answer_list = answer_corpus[0].split(".")
print(len(answer_list))

364


In [19]:
# Step 8: Conducting sentiment analysis for each sentence in the answer corpus

classifier = pipeline('sentiment-analysis', model='ProsusAI/finbert')

In [21]:
sentiment_matrix = pd.DataFrame(None, columns=['Label', 'Score'])
sentiment_analysis_results = classifier(answer_list)

for idx, per_sentence_sentiment in enumerate(sentiment_analysis_results):
    sentiment_matrix.loc[idx, ['Label', 'Score']] = (per_sentence_sentiment['label'], per_sentence_sentiment['score'])


In [24]:
# Step 9: Segregating statements by their sentiment >= 0.8

positive_corpus = []
neutral_corpus = []
negative_corpus = []

filtered_sentiment_matrix = sentiment_matrix[sentiment_matrix["Score"] >= 0.8]

for _index in filtered_sentiment_matrix.index.tolist():
    if sentiment_matrix.loc[_index, "Label"] == "positive":
        positive_corpus.append(answer_list[_index])
    elif sentiment_matrix.loc[_index, "Label"] == "negative":
        negative_corpus.append(answer_list[_index])
    else:
        neutral_corpus.append(answer_list[_index])

relevant_sentence_pool = [positive_corpus, neutral_corpus, negative_corpus]
strong_sentences = pd.DataFrame(
    (each_corpus for each_corpus in itertools.zip_longest(*relevant_sentence_pool)),
    columns=['Positive', 'Neutral', 'Negative'])

strong_sentences

,Positive,Neutral,Negative
0,"On the positive side, again, we're seeing rea...","As a remainder, all participant lines ...","But then suddenly, we had the lockdown again,..."
1,"But again, on the positive side, we believe t...",And there will be an opportunity for you to a...,Pricing continues to be a challen...
2, The second bit is on the Light Commercial V...,Should you need assistance during the confere...,I've been mentioning that over the last few q...
3,"But we, at the moment, believe that with the ...",Please note that this conference is being rec...,"At the same time, what we have been doing is ..."
4,"So, LCV business has been doing well it has b...",I now hand the conference over to Mr,"So, practically if you look at it schools wer..."
...,...,...,...
187,None,"Yes, because at the level of volume, it is b...",None
188,None,"See, understand one thing, inflation Ashok Le...",None
189,None,"So, there are a multiple ways to do this",None
190,None,"So, if I may take that as the last question",None


# -------------------------------------------------------------------------------------------

In [15]:
# Idea: Try to retain context.
# 1. Parse through the content and bin Questions and their supposed answers (Questions and all answers 
# following the questions.)
# 2. For each question -> Perform sentiment analysis on the answer corpus.
# 3. Select the top sentiment sentences > 0.8
# 4. Cluster sentences based on the sentiment (positive - positive - neutral - positive)
# 5. Cluster sentences based on the sentiment (negative - negative - neutral - negative)

In [59]:
outlook_indexes = information_frame[information_frame['type_of_conversation'] == "Outlook"].index.tolist()
question_indexes = information_frame[information_frame['type_of_conversation'] == "Question"].index.tolist()
answer_indexes = information_frame[information_frame['type_of_conversation'] == "Answer"].index.tolist()
indexes_to_exclude = information_frame[information_frame['type_of_conversation'].isnull()].index.tolist()

In [60]:
question_indexes

[9, 11, 13, 15, 17, 19, 23]

In [67]:
# Create groups of Question and answers
qna_groups = []
answer_added = 0  # Counter for answer added into each section
grouping = []
for i, question_index in enumerate(question_indexes):
    # For concall summaries having a format of Questions and then the answers, 
    # we group all the questions and answers in two groups, respectively.
    try:
        if len(grouping) >= 1:  # Will only be non-empty if answers are not yet added
            grouping.append({"type": "question",
                             "index": question_index,
                             "paragraph": information_frame.loc[question_index]['GroupOfSentences'].strip()})
        else:
            grouping = [{"type": "question",
                         "index": question_index,
                         "paragraph": information_frame.loc[question_index]['GroupOfSentences'].strip()}]
    except (KeyError, NameError):
        grouping = [{"type": "question",
                     "index": question_index,
                     "paragraph": information_frame.loc[question_index]['GroupOfSentences'].strip()}]
    for j, answer_index in enumerate(answer_indexes):
        try:
            if (answer_index > question_index) and (answer_index < question_indexes[i+1]):
                grouping.append({"type": "answer", 
                                 "index": answer_index,
                                 "paragraph": information_frame.loc[answer_index]['GroupOfSentences'].strip()})
                answer_added += 1
        except IndexError:
            if answer_index > question_index:
                grouping.append({"type": "answer", 
                                 "index": answer_index,
                                 "paragraph": information_frame.loc[answer_index]['GroupOfSentences'].strip()})
                answer_added += 1
    if (len(grouping) > 1) and (answer_added > 0):
        qna_groups.append(grouping)
        grouping = []
print(qna_groups)

[[{'type': 'question', 'index': 9, 'paragraph': "Chairman Sir, I'm from Bombay. Sir Resolution No.8 is the QIP of Rs.500 crores, I request you to come with the rights issue rather than a QIP. The dividend of 50 paisa, that is 50%, I would like to know whether you prefer YES Bank model where to give high dividend and QIP regularly and one fine day everybody knows what happened. Another model is the Kotak Bank which gives less than one rupee dividend on Rs.1,700 share price. Rarely does a QIP and grow with its own funds. Pages 4 of annual report, in each of the five years, deposits are more than advances. If more deposits are coming, reduce the interest rate. Deposit grew by 9% where the advances grew by 7%. During the year Rs.41.7 lakhs of dividend and 14,709 shares were transferred   to   IEPF.   It's   the   shareholders   money.   You   should   try   to   identify   the   genuine shareholder. Even if you have to send someone from the nearest branch on 28th September next month Rs.64

In [71]:
list(filter(lambda group: group["type"] == "question", qna_groups[0]))

[{'type': 'question',
  'index': 9,
  'paragraph': "Chairman Sir, I'm from Bombay. Sir Resolution No.8 is the QIP of Rs.500 crores, I request you to come with the rights issue rather than a QIP. The dividend of 50 paisa, that is 50%, I would like to know whether you prefer YES Bank model where to give high dividend and QIP regularly and one fine day everybody knows what happened. Another model is the Kotak Bank which gives less than one rupee dividend on Rs.1,700 share price. Rarely does a QIP and grow with its own funds. Pages 4 of annual report, in each of the five years, deposits are more than advances. If more deposits are coming, reduce the interest rate. Deposit grew by 9% where the advances grew by 7%. During the year Rs.41.7 lakhs of dividend and 14,709 shares were transferred   to   IEPF.   It's   the   shareholders   money.   You   should   try   to   identify   the   genuine shareholder. Even if you have to send someone from the nearest branch on 28th September next month Rs

In [ ]:
summary_matrix = pd.DataFrame(None, columns=['Question', 'Answer'])
summaries = []
for i, each_group in enumerate(groups):
    summary_matrix.loc[i, 'Question'] = question_indexes[i]
    summary_matrix.loc[i, 'Answer'] = answer_indexes[]
    